### Importing Packages

In [57]:
import pandas as pd # data processing
import fnmatch # file searching
import os
import folium # visualizing weather stations
from folium.plugins import MarkerCluster
from datetime import datetime # converting int into datetime
import plotly.express as px

### Collecting all geographic data

In [43]:
# collecting all metadata with geographical informations
dfs = []
# going through all files
for filename in os.listdir('Daten/Wetterdaten'):
    if fnmatch.fnmatch(filename, 'tageswerte*'):
        for filenames in os.listdir(f'Daten/Wetterdaten/{filename}'):
            # files with "geographie" in it
            if fnmatch.fnmatch(filenames, "*Geographie*"):
                df = pd.read_csv(f"Daten/Wetterdaten/{filename}/{filenames}", encoding="unicode_escape", delimiter=";")
                # lower case, because of different labeling
                df.columns = df.columns.str.lower()
                dfs.append(df)
# all geographic data in one dataframe
geodaten = pd.concat(dfs)
geodaten = geodaten.reset_index(drop=True)
geodaten.info()
geodaten.head(200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   stations_id    86 non-null     int64  
 1   stationshoehe  86 non-null     float64
 2   geogr.breite   86 non-null     float64
 3   geogr.laenge   86 non-null     float64
 4   von_datum      86 non-null     int64  
 5   bis_datum      86 non-null     object 
 6   stationsname   86 non-null     object 
dtypes: float64(3), int64(2), object(2)
memory usage: 4.8+ KB


,stations_id,stationshoehe,geogr.breite,geogr.laenge,von_datum,bis_datum,stationsname
0,377,181.0,49.1017,7.9920,19261101,19390831,"Bergzabern, Bad"
1,377,181.0,49.1017,7.9920,19410101,19431231,"Bergzabern, Bad"
2,377,181.0,49.1017,7.9920,19470101,19650831,"Bergzabern, Bad"
3,377,185.0,49.1040,7.9967,19650901,19770623,"Bergzabern, Bad"
4,377,180.0,49.1014,8.0034,19770624,20021231,"Bergzabern, Bad"
...,...,...,...,...,...,...,...
81,5717,178.0,51.2504,7.1455,19241001,19320331,Wuppertal-Buchenhofen
82,5717,126.0,51.2300,7.1100,19320401,20060831,Wuppertal-Buchenhofen
83,5717,134.0,51.2256,7.1052,20060901,,Wuppertal-Buchenhofen
84,5792,2956.0,47.4210,10.9848,19000719,20071016,Zugspitze


### Turning von_datum and bis_datum into datetime types

In [44]:
def zahl_zu_datum(zahl):
        try:
            datum_string = str(zahl)
            datum_datetime = datetime.strptime(datum_string, '%Y%m%d')
            datum_datetime2 = datetime.strftime(datum_datetime, '%m.%d.%Y')
            return datum_datetime2
        except:
            return None

geodaten['von_datum'] = geodaten['von_datum'].apply(zahl_zu_datum)
geodaten['von_datum'] = pd.to_datetime(geodaten["von_datum"])

geodaten["bis_datum"] = geodaten["bis_datum"].apply(zahl_zu_datum)
geodaten['bis_datum'] = pd.to_datetime(geodaten["bis_datum"])

geodaten.head(20)


,stations_id,stationshoehe,geogr.breite,geogr.laenge,von_datum,bis_datum,stationsname
0,377,181.00,49.1017,7.9920,1926-11-01,1939-08-31,"Bergzabern, Bad"
1,377,181.00,49.1017,7.9920,1941-01-01,1943-12-31,"Bergzabern, Bad"
2,377,181.00,49.1017,7.9920,1947-01-01,1965-08-31,"Bergzabern, Bad"
3,377,185.00,49.1040,7.9967,1965-09-01,1977-06-23,"Bergzabern, Bad"
4,377,180.00,49.1014,8.0034,1977-06-24,2002-12-31,"Bergzabern, Bad"
5,377,210.00,49.1070,7.9967,2003-03-11,NaT,"Bergzabern, Bad"
6,433,47.00,52.4641,13.4003,1928-01-01,1943-12-31,Berlin-Tempelhof
7,433,48.00,52.4705,13.4041,1944-01-01,1945-02-28,Berlin-Tempelhof
8,433,48.00,52.4705,13.4041,1945-03-01,1947-06-30,Berlin-Tempelhof
9,433,48.20,52.4686,13.4039,1947-07-01,1951-02-28,Berlin-Tempelhof


In [45]:
geodaten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   stations_id    86 non-null     int64         
 1   stationshoehe  86 non-null     float64       
 2   geogr.breite   86 non-null     float64       
 3   geogr.laenge   86 non-null     float64       
 4   von_datum      86 non-null     datetime64[ns]
 5   bis_datum      71 non-null     datetime64[ns]
 6   stationsname   86 non-null     object        
dtypes: datetime64[ns](2), float64(3), int64(1), object(1)
memory usage: 4.8+ KB


### Deleting all data before 1994

In [46]:
geodaten = geodaten[(geodaten['bis_datum'] >= pd.to_datetime('1994-01-01')) | (geodaten["bis_datum"].isna())]
geodaten.head(500)

,stations_id,stationshoehe,geogr.breite,geogr.laenge,von_datum,bis_datum,stationsname
4,377,180.00,49.1014,8.0034,1977-06-24,2002-12-31,"Bergzabern, Bad"
5,377,210.00,49.1070,7.9967,2003-03-11,NaT,"Bergzabern, Bad"
13,433,47.72,52.4675,13.4021,1970-09-10,2022-09-12,Berlin-Tempelhof
14,433,47.74,52.4676,13.4020,2022-09-13,NaT,Berlin-Tempelhof
20,555,77.00,51.4917,7.2157,1979-01-01,1994-04-30,Bochum
21,555,101.00,51.4789,7.2697,2007-03-19,2019-03-31,Bochum
22,555,110.00,51.5026,7.2289,2019-04-01,NaT,Bochum
24,850,39.00,52.5958,10.0280,1979-01-01,2007-06-13,Celle
25,850,39.00,52.5959,10.0296,2007-06-14,2017-03-30,Celle
26,850,44.74,52.5959,10.0296,2017-03-31,NaT,Celle


In [47]:
geodaten.to_csv("test.csv")

### Using the average of the station postition

In [48]:
mittelpunkt_stationen_br = geodaten.groupby("stationsname")["geogr.breite"].mean()
mittelpunkt_stationen_la = geodaten.groupby("stationsname")["geogr.laenge"].mean()
stations_mittelwert = geodaten.groupby("stationsname")["stationshoehe"].median()

stations_id = geodaten.loc[:, ["stations_id", "stationsname"]].drop_duplicates()

mittelpunkt_koordinaten = pd.merge(mittelpunkt_stationen_br,mittelpunkt_stationen_la, on="stationsname")
mittelpunkt_koordinaten = pd.merge(mittelpunkt_koordinaten, stations_mittelwert, on="stationsname")
mittelpunkt_koordinaten = pd.merge(mittelpunkt_koordinaten, stations_id, on="stationsname")

mittelpunkt_koordinaten.reset_index(drop=True, inplace=True)
mittelpunkt_koordinaten.to_csv("geographie.csv")

mittelpunkt_koordinaten


,stationsname,geogr.breite,geogr.laenge,stationshoehe,stations_id
0,"Bergzabern, Bad",49.104200,8.000050,195.00,377
1,Berlin-Tempelhof,52.467550,13.402050,47.73,433
2,Bochum,51.491067,7.238100,101.00,555
3,Celle,52.595867,10.029067,39.00,850
4,Cottbus,51.775900,14.316800,68.66,880
5,Freiburg,48.012000,7.839750,252.73,1443
6,Greifswald,54.096533,13.405633,2.00,1757
7,Hamburg-Fuhlsbüttel,53.634867,9.990500,11.00,1975
8,Norderney,53.712300,7.151900,11.47,3631
9,Pfullendorf,47.927500,9.287600,633.75,3927


In [49]:
median_hoehe = stations_mittelwert.median()
durschnitt_hoehe = round(stations_mittelwert.mean(),2)
print(median_hoehe)
print(durschnitt_hoehe)

130.0
357.76


In [50]:
median_breite = mittelpunkt_stationen_br.median()
median_laenge = mittelpunkt_stationen_la.median()
durschnitt_laenge = round(mittelpunkt_stationen_la.mean(),4)
durschnitt_breite = round(mittelpunkt_stationen_br.mean(),4)
median_breite

51.2278

### Plotting the locations of the weather stations

In [51]:
map_deutschland = folium.Map(location=[51.1657, 10.4515], zoom_start=6)

# Marker Cluster erstellen
marker_cluster = MarkerCluster().add_to(map_deutschland)

# Wetterstationen hinzufügen
for index, row in mittelpunkt_koordinaten.iterrows():
    folium.Marker(location=[row['geogr.breite'], row['geogr.laenge']],
                  popup=row['stationsname']).add_to(marker_cluster)
    

folium.Marker(location=[median_breite, median_laenge],
                  popup="Median").add_to(map_deutschland)

folium.Marker(location=[durschnitt_breite, durschnitt_laenge],
                  popup="Durchschnitt").add_to(map_deutschland)

map_deutschland

In [52]:

ids = mittelpunkt_koordinaten["stations_id"].to_list()
dfs = []
for ide in ids:
    filename = f"Daten/test/{ide}_produkt_klima_monat.csv"
    df = pd.read_csv(filename)
    neue_spalte = [ide] * len(df)
    df['stations_id'] = neue_spalte
    df['Datum'] = pd.to_datetime(df["Datum"])
    df = df[df['Datum'] >= pd.to_datetime('1994-01-01')]
    dfs.append(df)
wetterdaten = pd.concat(dfs)
wetterdaten = wetterdaten.reset_index(drop=True)
wetterdaten


,Datum,RSK,RSKF,SDK,SHK_TAG,NM,VPM,TMK,UPM,TXK,TNK,TGK,stations_id,FX,FM,PM
0,1994-01-01,3.390323,3.870968,1.816129,0.032258,4.851613,7.167742,3.845161,87.161290,6.396774,1.545161,-0.419355,377,NaN,NaN,NaN
1,1994-02-01,2.407143,3.071429,2.578571,0.535714,5.164286,6.225000,2.285714,81.785714,5.325000,-0.307143,-2.035714,377,NaN,NaN,NaN
2,1994-03-01,2.241935,2.709677,3.880645,0.000000,4.322581,9.077419,8.703226,79.580645,12.612903,5.551613,3.319355,377,NaN,NaN,NaN
3,1994-04-01,2.910000,3.533333,5.583333,0.000000,4.236667,8.926667,9.090000,76.400000,13.713333,4.943333,2.756667,377,NaN,NaN,NaN
4,1994-05-01,1.635484,3.290323,6.416129,0.000000,4.367742,11.854839,14.029032,73.806452,18.719355,9.796774,7.877419,377,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4961,2023-08-01,8.570968,5.806452,5.772000,2.806452,5.658065,7.396774,4.577419,86.313871,7.370968,1.935484,NaN,5792,15.212903,5.093548,714.832903
4962,2023-09-01,2.943333,2.600000,8.173867,3.566667,3.900000,5.943333,5.503333,68.826667,8.370000,3.170000,NaN,5792,12.760000,4.583333,716.819000
4963,2023-10-01,2.703226,4.419355,5.932839,3.258065,5.764516,4.412903,0.451613,70.833226,2.880645,-2.151613,NaN,5792,19.474194,7.861290,709.790323
4964,2023-11-01,13.213333,6.933333,2.453867,108.666667,6.496667,2.941379,-8.673333,87.506552,-5.690000,-11.500000,NaN,5792,24.790000,9.170000,699.255333


In [54]:
data = pd.merge(wetterdaten, mittelpunkt_koordinaten, on="stations_id")
data

,Datum,RSK,RSKF,SDK,SHK_TAG,NM,VPM,TMK,UPM,TXK,TNK,TGK,stations_id,FX,FM,PM,stationsname,geogr.breite,geogr.laenge,stationshoehe
0,1994-01-01,3.390323,3.870968,1.816129,0.032258,4.851613,7.167742,3.845161,87.161290,6.396774,1.545161,-0.419355,377,NaN,NaN,NaN,"Bergzabern, Bad",49.1042,8.00005,195.0
1,1994-02-01,2.407143,3.071429,2.578571,0.535714,5.164286,6.225000,2.285714,81.785714,5.325000,-0.307143,-2.035714,377,NaN,NaN,NaN,"Bergzabern, Bad",49.1042,8.00005,195.0
2,1994-03-01,2.241935,2.709677,3.880645,0.000000,4.322581,9.077419,8.703226,79.580645,12.612903,5.551613,3.319355,377,NaN,NaN,NaN,"Bergzabern, Bad",49.1042,8.00005,195.0
3,1994-04-01,2.910000,3.533333,5.583333,0.000000,4.236667,8.926667,9.090000,76.400000,13.713333,4.943333,2.756667,377,NaN,NaN,NaN,"Bergzabern, Bad",49.1042,8.00005,195.0
4,1994-05-01,1.635484,3.290323,6.416129,0.000000,4.367742,11.854839,14.029032,73.806452,18.719355,9.796774,7.877419,377,NaN,NaN,NaN,"Bergzabern, Bad",49.1042,8.00005,195.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4961,2023-08-01,8.570968,5.806452,5.772000,2.806452,5.658065,7.396774,4.577419,86.313871,7.370968,1.935484,NaN,5792,15.212903,5.093548,714.832903,Zugspitze,47.4210,10.98480,2956.0
4962,2023-09-01,2.943333,2.600000,8.173867,3.566667,3.900000,5.943333,5.503333,68.826667,8.370000,3.170000,NaN,5792,12.760000,4.583333,716.819000,Zugspitze,47.4210,10.98480,2956.0
4963,2023-10-01,2.703226,4.419355,5.932839,3.258065,5.764516,4.412903,0.451613,70.833226,2.880645,-2.151613,NaN,5792,19.474194,7.861290,709.790323,Zugspitze,47.4210,10.98480,2956.0
4964,2023-11-01,13.213333,6.933333,2.453867,108.666667,6.496667,2.941379,-8.673333,87.506552,-5.690000,-11.500000,NaN,5792,24.790000,9.170000,699.255333,Zugspitze,47.4210,10.98480,2956.0


In [58]:
corr_columns = ["geogr.breite", "geogr.laenge", "stationshoehe", "TMK"]
corr_data = data[corr_columns]
corr_data.corr()
corr = round(corr_data.corr(),2)

fig2 = px.imshow(corr,
                x=corr.columns,
                y=corr.columns,
                color_continuous_scale='Spectral',
                title='Correlation matrix',
                text_auto='True',
                width=500,
                height=500)
fig2.show()